## Figure 5: Climate baseline

In [ ]:
# spatial libraries
import shapely.geometry
import rioxarray as rioxr
import geopandas as gpd
import xarray as xr
import xesmf as xe
import regionmask

from xclim.indicators import atmos
from xclim import core 

import pandas as pd
import numpy as np
import os

# plotting libraries
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

# colors
cl = px.colors.qualitative.D3
cs = px.colors.sequential.Sunset

os.chdir('/home/rooda/')

In [ ]:
# Catchment shapefiles
basins = gpd.read_file("Dropbox/Patagonia/GIS South/Basins_Patagonia_ice.shp")
basins = basins.set_index("ID")

names = ["Yelcho", "Baker", "Santa Cruz                           ", "Palena", "Grey", "Puelo", "Cisnes", "Aysen", "Pascua"] # the space is important for visualization puroposes!
basins.loc[basins.basin_area > 5000, "Name"] = names

# Glacier shapefiles
rgi6 = gpd.read_file("Dropbox/Patagonia/GIS South/Glaciers/RGI6_v2.shp")[["geometry"]]
rgi7 = gpd.read_file("Dropbox/Patagonia/GIS South/Glaciers/RGI7_v2.shp")[["geometry"]]
glaciers  = pd.concat([rgi6.geometry, rgi7.geometry])
glaciers  = glaciers.buffer(0.05) # mask to use for baseline climate

In [ ]:
# Original (no regrid) reference climate (1980-2019): 
pp_pmet   = xr.open_dataset("OGGM_results/PMET_OGGM_1980_2019m.nc").prcp
pp_era5   = xr.open_dataset("OGGM_results/ERA5_OGGM_1980_2019m.nc").prcp
pp_cr2met = xr.open_dataset("OGGM_results/CR2MET_OGGM_1980_2019m.nc").prcp
pp_mswep  = xr.open_dataset("OGGM_results/MSWEP_OGGM_1980_2019m.nc").prcp

t2m_pmet   = xr.open_dataset("OGGM_results/PMET_OGGM_1980_2019m.nc").temp
t2m_era5   = xr.open_dataset("OGGM_results/ERA5_OGGM_1980_2019m.nc").temp
t2m_cr2met = xr.open_dataset("OGGM_results/CR2MET_OGGM_1980_2019m.nc").temp
t2m_mswep  = xr.open_dataset("OGGM_results/MSWEP_OGGM_1980_2019m.nc").temp

In [ ]:
# DEMs to downscale temperature
dem_005 = xr.open_dataset("OGGM_results/PMET_OGGM_1980_2019m.nc").hgt

dem_010 = xr.open_dataset("OGGM_results/MSWEP_OGGM_1980_2019m.nc").hgt
regridder  = xe.Regridder(dem_010,   dem_005, "nearest_s2d")
dem_010    = regridder(dem_010)

dem_025 = xr.open_dataset("OGGM_results/ERA5_OGGM_1980_2019m.nc").hgt
regridder  = xe.Regridder(dem_025,   dem_005, "nearest_s2d")
dem_025    = regridder(dem_025)

In [ ]:
# regrid (PMET as the reference grid; 0.05º)

## precipitation uses ESMF.RegridMethod.NEAREST_STOD
regridder  = xe.Regridder(pp_era5,   pp_pmet, "nearest_s2d")
pp_era5    = regridder(pp_era5)

regridder  = xe.Regridder(pp_cr2met, pp_pmet, "nearest_s2d")
pp_cr2met  = regridder(pp_cr2met)

regridder  = xe.Regridder(pp_mswep,  pp_pmet, "nearest_s2d")
pp_mswep   = regridder(pp_mswep)

## temperature uses lapse rate
lapse_rate = 0.0065 

regridder  = xe.Regridder(t2m_era5,   t2m_pmet, "nearest_s2d")
t2m_era5   = regridder(t2m_era5) # fake high res
factor     = (dem_025 - dem_005)*lapse_rate
t2m_era5   =  t2m_era5 + factor # "real" high res

regridder  = xe.Regridder(t2m_mswep,   t2m_pmet, "nearest_s2d")
t2m_mswep  = regridder(t2m_mswep) # fake high res
factor     = (dem_010 - dem_005)*lapse_rate
t2m_mswep  =  t2m_mswep + factor # "real" high res

regridder  = xe.Regridder(t2m_cr2met, t2m_pmet, "bilinear")
t2m_cr2met = regridder(t2m_cr2met) # simple case (same resolution)

In [ ]:
# mask: only glaciarated area
mask    = regionmask.mask_geopandas(glaciers, pp_pmet)   >= 0

pp_pmet    = pp_pmet.where(mask, drop = True)
pp_era5    = pp_era5.where(mask, drop = True)
pp_cr2met  = pp_cr2met.where(mask, drop = True)
pp_mswep   = pp_mswep.where(mask, drop = True)

t2m_pmet   = t2m_pmet.where(mask, drop = True)
t2m_era5   = t2m_era5.where(mask, drop = True)
t2m_cr2met = t2m_cr2met.where(mask, drop = True)
t2m_mswep  = t2m_mswep.where(mask, drop = True)

In [ ]:
# Calculate more variables

# xclim needs the units
pp_pmet.attrs['units']   = "mm month-1"
pp_era5.attrs['units']   = "mm month-1"
pp_cr2met.attrs['units'] = "mm month-1"
pp_mswep.attrs['units']  = "mm month-1"

t2m_pmet.attrs['units']   = "C"
t2m_era5.attrs['units']   = "C"
t2m_cr2met.attrs['units'] = "C"
t2m_mswep.attrs['units']  = "C"

# Positive degree day (month) sum (PDD)
ppd_pmet   = t2m_pmet.where(t2m_pmet >= -1, 0)
ppd_era5   = t2m_era5.where(t2m_era5 >= -1, 0)
ppd_cr2met = t2m_cr2met.where(t2m_cr2met >= -1, 0)
ppd_mswep  = t2m_mswep.where(t2m_mswep >= -1, 0)

# snowfall component
prsn_pmet = atmos.snowfall_approximation(pp_pmet, t2m_pmet, method='brown', thresh='0 degC')
prsn_pmet = core.units.convert_units_to(prsn_pmet, target = 'mm month-1', context = "hydro")
prsn_era5 = atmos.snowfall_approximation(pp_era5, t2m_era5, method='brown', thresh='0 degC')
prsn_era5 = core.units.convert_units_to(prsn_era5, target = 'mm month-1', context = "hydro")
prsn_cr2met = atmos.snowfall_approximation(pp_cr2met, t2m_cr2met, method='brown', thresh='0 degC')
prsn_cr2met = core.units.convert_units_to(prsn_cr2met, target = 'mm month-1', context = "hydro")
prsn_mswep = atmos.snowfall_approximation(pp_mswep, t2m_mswep, method='brown', thresh='0 degC')
prsn_mswep = core.units.convert_units_to(prsn_mswep, target = 'mm month-1', context = "hydro")

In [ ]:
# annual value
pp_pmet    = pp_pmet.resample(time='1Y').sum(skipna = False).mean(dim="time")
pp_era5    = pp_era5.resample(time='1Y').sum(skipna = False).mean(dim="time")
pp_cr2met  = pp_cr2met.resample(time='1Y').sum(skipna = False).mean(dim="time")
pp_mswep   = pp_mswep.resample(time='1Y').sum(skipna = False).mean(dim="time")

prsn_pmet    = prsn_pmet.resample(time='1Y').sum(skipna = False).mean(dim="time")
prsn_era5    = prsn_era5.resample(time='1Y').sum(skipna = False).mean(dim="time")
prsn_cr2met  = prsn_cr2met.resample(time='1Y').sum(skipna = False).mean(dim="time")
prsn_mswep   = prsn_mswep.resample(time='1Y').sum(skipna = False).mean(dim="time")

t2m_pmet   = t2m_pmet.resample(time='1Y').mean(skipna = False).mean(dim="time")
t2m_era5   = t2m_era5.resample(time='1Y').mean(skipna = False).mean(dim="time")
t2m_cr2met = t2m_cr2met.resample(time='1Y').mean(skipna = False).mean(dim="time")
t2m_mswep  = t2m_mswep.resample(time='1Y').mean(skipna = False).mean(dim="time")

ppd_pmet   = ppd_pmet.resample(time='1Y').sum(skipna = False).mean(dim="time")
ppd_era5   = ppd_era5.resample(time='1Y').sum(skipna = False).mean(dim="time")
ppd_cr2met = ppd_cr2met.resample(time='1Y').sum(skipna = False).mean(dim="time")
ppd_mswep  = ppd_mswep.resample(time='1Y').sum(skipna = False).mean(dim="time")

In [ ]:
# mean value for each catchment
averager   = xe.SpatialAverager(pp_pmet,   basins.geometry, geom_dim_name="avg")

basins["PP_PMET"]   = averager(pp_pmet,   skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PP_ERA5"]   = averager(pp_era5,   skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PP_CR2MET"] = averager(pp_cr2met, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PP_MSWEP"]  = averager(pp_mswep,  skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values

basins["PRSN_PMET"]   = averager(prsn_pmet,   skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PRSN_ERA5"]   = averager(prsn_era5,   skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PRSN_CR2MET"] = averager(prsn_cr2met, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PRSN_MSWEP"]  = averager(prsn_mswep,  skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values

basins["T2M_PMET"]   = averager(t2m_pmet,   skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_ERA5"]   = averager(t2m_era5,   skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_CR2MET"] = averager(t2m_cr2met, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["T2M_MSWEP"]  = averager(t2m_mswep,  skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values

basins["PPD_PMET"]   = averager(ppd_pmet,   skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PPD_ERA5"]   = averager(ppd_era5,   skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PPD_CR2MET"] = averager(ppd_cr2met, skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values
basins["PPD_MSWEP"]  = averager(ppd_mswep,  skipna=True).assign_coords(avg=xr.DataArray(basins.index, dims=("avg",))).values

# major catchments
basins_m = basins.dropna(subset = ['Name'])

In [ ]:
# basemap for background
geo_map = gpd.read_file("Dropbox/ArcGIS/Chile/south_america.shp")
geo_map = geo_map[(geo_map.CC == "CI") | (geo_map.CC == "AR")]
geo_map = geo_map.dissolve(by='REGION')
geo_map["geometry"] = geo_map.simplify(0.01)

poly_gdf = shapely.geometry.Polygon([(-76, -55.7), (-76, -40.52), (-68.05, -40.52), (-68.05, -55.7), (-76, -55.8)])
poly_gdf = gpd.GeoDataFrame([1], geometry=[poly_gdf], crs=geo_map.crs)

geo_map = geo_map.clip(poly_gdf)

In [ ]:
# hydrological zone divides
geo_lines = gpd.read_file("Dropbox/Patagonia/GIS South/Basins_Patagonia_ice_divides.shp")

lats = []
lons = []

for feature in geo_lines.geometry:
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        
lat_coords = [-43.2, -45.95,  -46.4,  -47.55,  -49.2,   -50.5,   -52.0, -53.1, -54.8]
lon_coords = [-71.2, -71.7,   -74.5,  -71.7,   -72.2,   -72.3,   -72.1, -71.7, -68.9]
names      = ["PPY", "PCA", "NPI-W", "NPI-E", "SPI-N", "SPI-C", "SPI-S", "GCN", "CDI"]
names  = ['<b>'+x+'</b>' for x in names]

In [ ]:
fig = make_subplots(rows=3, cols=3, horizontal_spacing = 0.01, vertical_spacing = 0.02, column_widths = [0.36, 0.36, 0.31], shared_xaxes = True,
                    subplot_titles = ["a) Mean annual precipitation (PMET)","b) Mean annual temperature (PMET)", "Relative difference to PMET"],
                    specs=[[{"type": "scattergeo", "rowspan": 3}, {"type": "scattergeo", "rowspan": 3}, {"type": "histogram"}],
                           [          None,                                      None,                  {"type": "histogram"}],
                           [          None,                                      None,                  {"type": "histogram"}]])

## Basemap
for x in range(1,3):
    fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                            colorscale = ["#d5d5d5", "#d5d5d5"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=1, col=x)


# Precipitation mean change (a) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()),  locations = basins.index, z = basins['PP_PMET'], 
                            colorscale = ["#ccebc5", "#4eb3d3", "#034b8a"], marker_line_color ='white', marker_line_width=0.1, 
                            zmin = 1000, zmax = 8000, colorbar=dict(len=0.45, x=0.24, y= 0.77, title='Precipitation', ticksuffix = " mm", thickness=20)), row=1, col=1)

# Temperature mean change (b) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()), locations = basins.index, z = basins['T2M_PMET'], 
                            colorscale=[cs[5], cs[3],  cs[0]], marker_line_color='white', marker_line_width=0.1, 
                            zmin = -2, zmax = 8, colorbar=dict(len=0.45, x=0.59, y= 0.77, title='Temperature', ticksuffix = " ºC", thickness=20)), row=1, col=2)


# layout a) and b) ---------------------------------------------------------------------------------------------------------------------
for x in range(1,3):
    ## Add basin and hydrological zone names plus the hydro zone divides
    fig.add_trace(go.Scattergeo(lon = lons, lat = lats, mode = 'lines', line = dict(width = 0.7,color = 'black'),opacity = 0.5, showlegend = False),row=1, col=x)  
    fig.add_trace(go.Scattergeo(lon = lon_coords, lat=lat_coords, mode='text', text=names, textfont=dict(size=12, color = "rgba(0,0,0,0.5)"), showlegend = False),row=1, col=x)
    fig.add_scattergeo(geojson = eval(basins['geometry'].to_json()), locations = basins.index, text = basins['Name'], mode = 'text', showlegend = False,
                       textfont=dict(size=11, color = "rgba(0,0,0,0.3)"),row=1, col=x)

fig.update_geos(showframe = True, framewidth = 1,  framecolor = "black", lonaxis_range=[-76, -68], lataxis_range=[-55.8, -40.5], 
                bgcolor = "#f9f9f9", showland = False, showcoastlines = False, showlakes = False)

# Solid precipitation spread (c) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Violin(y = ((basins.PP_CR2MET/basins.PP_PMET)-1).values, marker_color = cl[0], name = "CR2MET", opacity=0.7, points = False, showlegend = False), row=1, col=3)
fig.add_trace(go.Violin(y = ((basins_m.PP_CR2MET/basins_m.PP_PMET)-1).values,  marker_color= cl[0], line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all", name = "CR2MET",  showlegend = False), row=1, col=3)
fig.add_trace(go.Violin(y = ((basins.PP_ERA5/basins.PP_PMET)-1).values,   marker_color = cl[1], name = "ERA5", opacity=0.7, points = False, showlegend = False), row=1, col=3)
fig.add_trace(go.Violin(y = ((basins_m.PP_ERA5/basins_m.PP_PMET)-1).values,  marker_color= cl[1], line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all", name = "ERA5",  showlegend = False), row=1, col=3)
fig.add_trace(go.Violin(y = ((basins.PP_MSWEP/basins.PP_PMET)-1).values,  marker_color = cs[5], name = "MSWEP", opacity=0.7, points = False, showlegend = False), row=1, col=3)
fig.add_trace(go.Violin(y = ((basins_m.PP_MSWEP/basins_m.PP_PMET)-1).values,  marker_color= cs[5], line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all", name = "MSWEP",  showlegend = False), row=1, col=3)
fig.update_traces(box_visible=True, width=0.7, meanline_visible=True, row = 1, col = 3)

fig.update_yaxes(title_text="Δ Solid precpitation (%)", side = "right", title_standoff = 2, row = 1, col = 3)
fig.update_yaxes(range = [-0.8,0.8], tickangle = 0, tickformat = ',.0%', row = 1, col = 3)
fig.add_annotation(text="c)", font=dict(size=16), x=-0.65, y=0.7, showarrow=False, row=1, col=3)
fig.add_annotation(text="Main catchments", font=dict(size=14), ax=30, x=-0.5, y=0.15, showarrow=True, row=1, col=3)

# Positive degree day (month) sum spread (d) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Violin(y = ((basins.PPD_CR2MET/basins.PPD_PMET)-1).values, marker_color= cl[0], name = "CR2MET", opacity=0.7, points = False, showlegend = False), row=2, col=3)
fig.add_trace(go.Violin(y = ((basins_m.PPD_CR2MET/basins_m.PPD_PMET)-1).values,  marker_color= cl[0], line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all", name = "CR2MET",  showlegend = False), row=2, col=3)
fig.add_trace(go.Violin(y = ((basins.PPD_ERA5/basins.PPD_PMET)-1).values,   marker_color= cl[1], name = "ERA5", opacity=0.7, points = False, showlegend = False), row=2, col=3)
fig.add_trace(go.Violin(y = ((basins_m.PPD_ERA5/basins_m.PPD_PMET)-1).values,  marker_color= cl[1], line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all", name = "ERA5",  showlegend = False), row=2, col=3)
fig.add_trace(go.Violin(y = ((basins.PPD_MSWEP/basins.PPD_PMET)-1).values,  marker_color= cs[5], name = "MSWEP", opacity=0.7, points = False, showlegend = False), row=2, col=3)
fig.add_trace(go.Violin(y = ((basins_m.PPD_MSWEP/basins_m.PPD_PMET)-1).values,  marker_color= cs[5], line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all", name = "MSWEP",  showlegend = False), row=2, col=3)
fig.update_traces(box_visible=True, width=0.7, meanline_visible=True, row = 2, col = 3)

fig.update_yaxes(title_text="Δ Positive degree month sum (%)", side = "right", title_standoff = 0, row = 2, col = 3)
fig.update_yaxes(range = [-0.8,0.8], tickangle = 0, tickformat = ',.0%', row = 2, col = 3)
fig.add_annotation(text="d)", font=dict(size=16), x=-0.65, y=0.7, showarrow=False, row=2, col=3)

# Temperature spread (e) -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Violin(y = (basins.T2M_CR2MET-basins.T2M_PMET).values, marker_color= cl[0], opacity=0.7, points = False, name = "CR2MET", showlegend = False), row=3, col=3)
fig.add_trace(go.Violin(y = (basins_m.T2M_CR2MET-basins_m.T2M_PMET).values,  marker_color=cl[0], line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all", name = "CR2MET",  showlegend = False), row=3, col=3)
fig.add_trace(go.Violin(y = (basins.T2M_ERA5-basins.T2M_PMET).values,   marker_color= cl[1], opacity=0.7, points = False, name = "ERA5",   showlegend = False), row=3, col=3)
fig.add_trace(go.Violin(y = (basins_m.T2M_ERA5-basins_m.T2M_PMET).values,  marker_color=cl[1], line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all", name = "ERA5",  showlegend = False), row=3, col=3)
fig.add_trace(go.Violin(y = (basins.T2M_MSWEP-basins.T2M_PMET).values,  marker_color= cs[5], opacity=0.7, points = False, name = "MSWEP",  showlegend = False), row=3, col=3)
fig.add_trace(go.Violin(y = (basins_m.T2M_MSWEP-basins_m.T2M_PMET).values,  marker_color=cs[5], line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all", name = "MSWEP",  showlegend = False), row=3, col=3)
fig.update_traces(box_visible=True, width=0.7, meanline_visible=True, row = 3, col = 3)

fig.update_yaxes(title_text="Δ Air temperature (ºC)", side = "right", title_standoff = 20, row = 3, col = 3)
fig.update_yaxes(range = [-4, 4], tickangle = 0, row = 3, col = 3)
fig.add_annotation(text="e)", font=dict(size=16), x=-0.65, y=3.4, showarrow=False, row=3, col=3)


# layout c, d and e
fig.update_layout(plot_bgcolor="rgba(213,213,213,0.6)")
fig.update_xaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.8)", zeroline=False, showline = True, linecolor = 'black', linewidth = 1, ticks="outside", mirror=True)
fig.update_yaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.8)", zeroline=True,  showline = True, linecolor = 'black', linewidth = 1, ticks="outside", mirror=True)

# general 
fig.update_layout(autosize = False, width = 1000, height = 670, margin = dict(l=10, r=5, b=5, t=30, pad=0, autoexpand=True))

fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_5_climate.png", scale=4)
fig.show()